# 实例-搭建疫情数据分析网站

## 0 实例介绍

## 1 获取数据

## 2 网站`html`文件编辑

## 3 基础图像的绘制

本项目中，我们仍然使用`covid19`数据集，该数据集收集了31个身份53天内的疫情数据，我们参考JHU疫情分析以及百度大数据疫情分析网站，来制作基础的图像。

In [8]:
import pandas as pd 
import numpy as np 
from bokeh.plotting import figure, output_file, show
from bokeh.plotting import ColumnDataSource as CDS 

In [5]:
covid19 = pd.read_csv('covid19.csv')
covid19['Observation Date'] = pd.to_datetime(covid19['Observation Date'], format = '%d-%m-%Y')

In [15]:
provincedata = covid19.groupby('Province/State')[['Confirmed', 'Observation Date']]

In [23]:
covid19

,Province/State,Country/Region,Confirmed,Deaths,Recovered,Latitude,Longitude,Observation Date
0,Anhui,Mainland China,1,0,0,32.0,117.0,2020-01-22
1,Anhui,Mainland China,9,0,0,32.0,117.0,2020-01-23
2,Anhui,Mainland China,15,0,0,32.0,117.0,2020-01-24
3,Anhui,Mainland China,39,0,0,32.0,117.0,2020-01-25
4,Anhui,Mainland China,60,0,0,32.0,117.0,2020-01-26
...,...,...,...,...,...,...,...,...
1661,Zhejiang,Mainland China,1215,1,1195,29.0,120.0,2020-03-11
1662,Zhejiang,Mainland China,1215,1,1197,29.0,120.0,2020-03-12
1663,Zhejiang,Mainland China,1215,1,1197,29.0,120.0,2020-03-13
1664,Zhejiang,Mainland China,1227,1,1211,29.0,120.0,2020-03-14


In [70]:
cityname = []
confirmed = []
deaths = []
latitude = []
longitude = []
date = list(provincedata)[0][1]['Observation Date']

for city in list(provincedata):
    cityname.append(city[0])
    confirmed.append(list(city[1]['Confirmed']))
    deaths.append(list(city[1]['Deaths']))
    latitude.append(city[1]['Latitude'].iloc[0])
    longitude.append(city[1]['Longitude'].iloc[0])

Confirmed = pd.DataFrame(pd.DataFrame(confirmed).values.T, index = date, columns = cityname)
Deaths = pd.DataFrame(pd.DataFrame(deaths).values.T, index = date, columns = cityname)

In [72]:
Confirmed

,Anhui,Beijing,Chongqing,Fujian,Gansu,Guangdong,Guangxi,Guizhou,Hainan,Hebei,Heilongjiang,Henan,Hubei,Hunan,Inner Mongolia,Jiangsu,Jiangxi,Jilin,Liaoning,Ningxia,Qinghai,Shaanxi,Shandong,Shanghai,Shanxi,Sichuan,Tianjin,Tibet,Xinjiang,Yunnan,Zhejiang
Observation Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2020-01-22,1.0,14.0,6.0,1.0,0.0,26.0,2.0,1.0,4.0,1.0,0.0,5.0,444.0,4.0,0.0,1.0,2.0,0.0,2.0,1.0,0.0,0.0,2.0,9.0,1.0,5.0,4.0,1.0,0.0,1.0,10.0
2020-01-23,9.0,22.0,9.0,5.0,2.0,32.0,5.0,3.0,5.0,1.0,2.0,5.0,444.0,9.0,0.0,5.0,7.0,1.0,3.0,1.0,0.0,3.0,6.0,16.0,1.0,8.0,4.0,1.0,2.0,2.0,27.0
2020-01-24,15.0,36.0,27.0,10.0,2.0,53.0,23.0,3.0,8.0,2.0,4.0,9.0,549.0,24.0,1.0,9.0,18.0,3.0,4.0,2.0,0.0,5.0,15.0,20.0,1.0,15.0,8.0,1.0,2.0,5.0,43.0
2020-01-25,39.0,41.0,57.0,18.0,4.0,78.0,23.0,4.0,19.0,8.0,9.0,32.0,761.0,43.0,7.0,18.0,18.0,4.0,17.0,3.0,1.0,15.0,27.0,33.0,6.0,28.0,10.0,1.0,3.0,11.0,62.0
2020-01-26,60.0,68.0,75.0,35.0,7.0,111.0,36.0,5.0,22.0,13.0,15.0,83.0,1058.0,69.0,7.0,33.0,36.0,4.0,21.0,4.0,1.0,22.0,46.0,40.0,9.0,44.0,14.0,1.0,4.0,16.0,104.0
2020-01-27,70.0,80.0,110.0,59.0,14.0,151.0,46.0,7.0,33.0,18.0,21.0,128.0,1423.0,100.0,11.0,47.0,72.0,6.0,27.0,7.0,6.0,35.0,75.0,53.0,13.0,69.0,23.0,1.0,5.0,26.0,128.0
2020-01-28,106.0,91.0,132.0,80.0,19.0,207.0,51.0,9.0,40.0,33.0,33.0,168.0,3554.0,143.0,15.0,70.0,109.0,8.0,34.0,11.0,6.0,46.0,95.0,66.0,27.0,90.0,24.0,1.0,10.0,44.0,173.0
2020-01-29,152.0,111.0,147.0,84.0,24.0,277.0,58.0,9.0,43.0,48.0,38.0,206.0,3554.0,221.0,16.0,99.0,109.0,9.0,39.0,12.0,6.0,56.0,130.0,96.0,27.0,108.0,27.0,1.0,13.0,55.0,296.0
2020-01-30,200.0,114.0,182.0,101.0,26.0,354.0,78.0,12.0,46.0,65.0,44.0,278.0,4903.0,277.0,19.0,129.0,162.0,14.0,41.0,17.0,8.0,63.0,158.0,112.0,35.0,142.0,31.0,1.0,14.0,70.0,428.0


In [86]:
source = CDS(Confirmed)

Tooltips = [
    ("Province:", "$name"),
    ('Date:', "$x"),
    ('Confirmed Infection:', '$y'),
]


p = figure(title = 'Daily Confirmed', x_axis_type = 'datetime', tooltips = Tooltips)

p.vline_stack(cityname, x = 'Observation Date', source = source, line_width = 2)

show(p)